<a href="https://colab.research.google.com/github/Tee-A/Tagui_Fetch_LinkedIN_URL/blob/main/Test_Python_RPA_Url_Fetch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up Environment For Tagui on Colab

In [1]:
# setup RPA environment by installing PHP, Chromium web browser and RPA for Python
!apt-get update; apt install php; apt install chromium-browser; pip install rpa

# run r.setup() to download TagUI, modify to run Chromium browser and as root user
import rpa as r; r.setup(); r.dump(r.load('/root/.tagui/src/tagui').replace('"google-chrome"', '"chromium-browser"').replace('$headless_switch', '--no-sandbox'), '/root/.tagui/src/tagui')

# virtual display to run web browser normally (some websites reject headless mode)
!apt install xvfb; pip install pyvirtualdisplay
import pyvirtualdisplay; display = pyvirtualdisplay.Display(); display.start()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

# IMporting other Libraries and Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rpa as r

# Reading dataset that conatins names and company names of users

In [3]:
df = pd.read_csv('/content/Users_test.csv')
df2 = df.replace(np.nan, '', regex=True)
df2.head()

,Name,Company
0,Richard Olayanju,Octave
1,Ntongha Ibiang,ParallelScore
2,"Victor Momodu, ABDA",
3,Edward Oluwaseyi Olayemi,First Bank
4,Sadeeq Akinyemi AKINTOLA,


In [4]:
df2['search'] = df2['Name'] +" " + df2['Company'] + ' site:linkedin.com'
df2.head(6)

,Name,Company,search
0,Richard Olayanju,Octave,Richard Olayanju Octave site:linkedin.com
1,Ntongha Ibiang,ParallelScore,Ntongha Ibiang ParallelScore site:linkedin.com
2,"Victor Momodu, ABDA",,"Victor Momodu, ABDA site:linkedin.com"
3,Edward Oluwaseyi Olayemi,First Bank,Edward Oluwaseyi Olayemi First Bank site:linke...
4,Sadeeq Akinyemi AKINTOLA,,Sadeeq Akinyemi AKINTOLA site:linkedin.com
5,Boluwatife Okusanya,ParallelScore,Boluwatife Okusanya ParallelScore site:linkedi...


In [5]:
query = df2['search'].to_list()
r.init()
for q in query:
    r.url('https://www.google.com')
    #r.wait(1)
    r.type('//*[@name="q"]', f'{q}[enter]')
    r.wait(1)
    l = r.read('.g:nth-child(1) .LC20lb.DKV0Md')
    #r.wait(1)
    link = r.read('.g:nth-child(1) .TbwUpd.NJjxre')
    #r.rclick('.g:nth-child(1) .LC20lb.DKV0Md')
    #r.keyboard('[E]')
    #r.wait(1)
    r.write(link + '\n','/content/profile_url.csv')
    r.wait(1)
    r.write(l + '\n', '/content/position.csv')
    r.wait(1)
    #r.keyboard('[backspace]')
    r.wait(1)
    

In [6]:
r.close()

True

In [7]:
profile = pd.read_csv('/content/profile_url.csv')
positions = pd.read_csv('/content/position.csv')
profile['links'] = profile['Link'].str.split(" › ").str[0] + "/in/" + profile['Link'].str.split(" › ").str[1]

In [10]:
profile.drop(['Link'],axis = 1, inplace = True)
frame = [positions,profile]
final_df = pd.concat(frame,axis=1)
#final_df.to_csv('profile_details.csv', index=False)
final_df = final_df.replace(np.nan, '', regex=True)

In [11]:
final_df.head(10)

,Name_Bio,Certification_Credential,links
0,Richard Olayanju,ABDA™ - Data Scientist - Octave Analytics,https://ng.linkedin.com/in/richard-olayanju
1,Ntongha Ibiang - NLP Data Scientist - Parallel...,,https://ng.linkedin.com/in/ntongha-ibiang-a0b9...
2,Victor Momodu,ABDA™ – Jacobs University Bremen - LinkedIn,https://de.linkedin.com/in/victormomodu
3,Edward Oluwaseyi Olayemi - Team Lead,Insight and Analytics,https://ng.linkedin.com/in/eddyena
4,Sadeeq Akinyemi AKINTOLA - Azure Cloud Big Dat...,,https://pt.linkedin.com/in/sadeeqakintola
5,Boluwatife Okusanya - Human Resource Manager -...,,https://ng.linkedin.com/in/okusanya-boluwatife
6,Opeoluwa Iwashokun - Country Manager - Hebroux...,,https://ng.linkedin.com/in/opeoluwa-iwashokun-...


# Applying Transformation

In [12]:
final_df['FullName'] = final_df['Name_Bio'].str.split('-').str[0]
final_df['Position'] = final_df['Name_Bio'].str.split('-').str[1]
final_df['Extra'] = final_df['Name_Bio'].str.split('-').str[2]
final_df['Position2'] = final_df['Certification_Credential'].str.split('-').str[1]
final_df['Certification_Credential'] = final_df['Certification_Credential'].str.split('-').str[0]
final_df = final_df.replace(np.nan, '', regex=True)
final_df['Bio_And_CertificationExtra'] = final_df['Certification_Credential'] + final_df['Extra']
final_df['Position'] = final_df['Position'] + final_df['Position2']
final_df['Linkedin_Location'] = final_df['links'].str.slice(8,10)
final_df = final_df.drop(['Extra','Position2','Certification_Credential'], axis = 1)
final_df.head(10)

,Name_Bio,links,FullName,Position,Bio_And_CertificationExtra,Linkedin_Location
0,Richard Olayanju,https://ng.linkedin.com/in/richard-olayanju,Richard Olayanju,Data Scientist,ABDA™,ng
1,Ntongha Ibiang - NLP Data Scientist - Parallel...,https://ng.linkedin.com/in/ntongha-ibiang-a0b9...,Ntongha Ibiang,NLP Data Scientist,ParallelScore,ng
2,Victor Momodu,https://de.linkedin.com/in/victormomodu,Victor Momodu,LinkedIn,ABDA™ – Jacobs University Bremen,de
3,Edward Oluwaseyi Olayemi - Team Lead,https://ng.linkedin.com/in/eddyena,Edward Oluwaseyi Olayemi,Team Lead,Insight and Analytics,ng
4,Sadeeq Akinyemi AKINTOLA - Azure Cloud Big Dat...,https://pt.linkedin.com/in/sadeeqakintola,Sadeeq Akinyemi AKINTOLA,Azure Cloud Big Data Support ...,,pt
5,Boluwatife Okusanya - Human Resource Manager -...,https://ng.linkedin.com/in/okusanya-boluwatife,Boluwatife Okusanya,Human Resource Manager,LinkedIn ...,ng
6,Opeoluwa Iwashokun - Country Manager - Hebroux...,https://ng.linkedin.com/in/opeoluwa-iwashokun-...,Opeoluwa Iwashokun,Country Manager,Hebroux Consulting,ng
